In [7]:
import pandas as pd
import numpy as np

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [9]:
df = pd.read_csv('covid_toy.csv')

In [11]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [13]:
X_train

,age,gender,fever,cough,city
5,84,Female,NaN,Mild,Bangalore
96,51,Female,101.0,Strong,Kolkata
65,69,Female,102.0,Mild,Bangalore
89,46,Male,103.0,Strong,Bangalore
27,33,Female,102.0,Strong,Delhi
...,...,...,...,...,...
34,74,Male,102.0,Mild,Mumbai
40,49,Female,102.0,Mild,Delhi
51,11,Female,100.0,Strong,Kolkata
97,20,Female,101.0,Mild,Bangalore


In [14]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

## 1. Without Column Transformer

In [15]:
# simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [19]:
# Ordinal Encoding for Cough Col
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [22]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [24]:
# Extrating Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [36]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## 2. Using Column Transformer

In [29]:
from sklearn.compose import ColumnTransformer

In [31]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [33]:
transformer.fit_transform(X_train).shape

(80, 7)

In [34]:
transformer.fit_transform(X_test).shape

(20, 7)